## Transit Bunching 
* `cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest`
* [Issue](https://github.com/cal-itp/data-analyses/issues/1099)
### 11/8
* Figure out how to address City of Visalia: one of the buses that is scheduled to arrive earlier arrives later than another bus. 
* This leads to a negative time stamp and makes it appear like there is a lot of bunching per the Transit Matters approach.



In [ ]:
import datetime as dt

import altair as alt
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

# https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
may_date = "2024-05-22"

In [ ]:
drop_for_preview = [
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "trip_id",
]

### Grab Sample Routes

In [ ]:
subset = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
    "service_date",
    "frequency",
]

In [ ]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

In [ ]:
route_dir_columns = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "time_period",
    "route_primary_direction",
    "frequency",
    "service_date",
]

In [ ]:
route_dir = merge_data.concatenate_schedule_by_route_direction([may_date])[
    route_dir_columns
]

In [ ]:
route_dir.head()

#### Attach operators and districts

In [ ]:
# Grab Crosswalk
CROSSWALK = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

In [ ]:
crosswalk_cols = [
    "schedule_gtfs_dataset_key",
    "organization_name",
    "name",
    "caltrans_district",
]

In [ ]:
crosswalk_df = (
    time_series_utils.concatenate_datasets_across_dates(
        SCHED_GCS, CROSSWALK, [may_date], data_type="df", columns=crosswalk_cols
    )
    .sort_values(["service_date"])
    .reset_index(drop=True)
)

In [ ]:
crosswalk_df.head(2)

In [ ]:
crosswalk_df.shape

In [ ]:
routes_dir2 = pd.merge(
    route_dir,
    crosswalk_df,
    on=["schedule_gtfs_dataset_key", "service_date"],
    how="left",
)

In [ ]:
thousand_oaks = routes_dir2.loc[
    (routes_dir2.organization_name == "City of Thousand Oaks")
    & (routes_dir2.route_id == "3402")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [ ]:
visalia = routes_dir2.loc[
    (routes_dir2.organization_name == "City of Visalia")
    & (routes_dir2.route_id == "2042")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [ ]:
metro = routes_dir2.loc[
    (
        routes_dir2.organization_name
        == "Los Angeles County Metropolitan Transportation Authority"
    )
    & (routes_dir2.route_id == "204-13172")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [ ]:
# '33-13172'
metro_33 = routes_dir2.loc[
    (
        routes_dir2.organization_name
        == "Los Angeles County Metropolitan Transportation Authority"
    )
    & (routes_dir2.route_id == "33-13172")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [ ]:
routes = pd.concat([thousand_oaks, visalia, metro, metro_33])

In [ ]:
routes.route_id.unique()

In [ ]:
routes.head(3)

### Add Trips

In [ ]:
TABLE = GTFS_DATA_DICT.schedule_downloads.trips

In [ ]:
FILE = f"{COMPILED_CACHED_VIEWS}{TABLE}_{may_date}.parquet"

In [ ]:
trips_subset = [
    "gtfs_dataset_key",
    "route_id",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "route_long_name",
    "direction_id",
    "route_type",
]

In [ ]:
trips = pd.read_parquet(FILE)[trips_subset].rename(
    columns={"gtfs_dataset_key": "schedule_gtfs_dataset_key"}
)

In [ ]:
trips_routes = pd.merge(
    trips,
    routes_dir2,
    on=["schedule_gtfs_dataset_key", "route_id", "direction_id"],
    how="inner",
)

In [ ]:
trips_routes.shape

In [ ]:
trips_routes.route_id.nunique()

#### I know we can get this from the warehouse but it seems cumbersome. Correct me if I'm wrong.

In [ ]:
# https://gtfs.org/documentation/schedule/reference/#
route_type_crosswalk = {
    "route_type": ["0", "1", "2", "3", "4", "5", "6", "7", "11", "12"],
    "route_type_str": [
        "Tram, Streetcar, Light rail",
        "Subway, Metro",
        "Rail",
        "Bus",
        "Ferry.",
        "Cable tram.",
        "Aerial lift, suspended cable car (e.g., gondola lift, aerial tramway).",
        "Funicular.",
        "Trolleybus.",
        "Monorail.",
    ],
}

In [ ]:
route_type_crosswalk_df = pd.DataFrame(route_type_crosswalk)

In [ ]:
# Merge for route_type
trips_routes = pd.merge(
    trips_routes, route_type_crosswalk_df, on=["route_type"], how="left"
)

In [ ]:
trips_routes = trips_routes.drop(columns=["route_type"])

In [ ]:
trips_routes = trips_routes.rename(columns={"route_type_str": "route_type"})

### Get Stop Times 

In [ ]:
rt_stop_times = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_rt_stop_times_2024-05-22.parquet"
)

In [ ]:
rt_stop_times.head(2)

In [ ]:
trips_routes_times = pd.merge(
    rt_stop_times,
    trips_routes,
    on=[
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
    ],
    how="inner",
)

In [ ]:
(trips_routes_times.scheduled_arrival_sec.isna().sum())

In [ ]:
trips_routes_times.info()

### Sorting 

In [ ]:
trips_routes_times2 = trips_routes_times.sort_values(
    by=[
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "shape_array_key",
        "direction_id",
        "stop_sequence",
        "stop_id",
        "rt_arrival_sec",
    ]
).reset_index(drop=True)

In [ ]:
preview_sort_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "stop_sequence",
    "rt_arrival_sec",
    "stop_id",
    "scheduled_arrival_sec",
]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.direction_id == 1)
][preview_sort_col].shape

### Convert scheduled and RT arrival times.
* If 82800  < `scheduled_arrival_time` < 86_400 but `rt_arrival_sec` is lower say 14_000 (4 am in the morning): then the bus was scheduled to arrive on May 21 (day before the service date) but it arrived a little later on the actual service date we query the data for. 
* If  86_400 < `scheduled_arrival_time` and `rt_arrival_sec` is around 86_000 then this is the same service date. 

In [ ]:
trips_routes_times2["rt_arrival_sec"].describe()

In [ ]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] == 86_400].shape

####  Filter out for trips with `scheduled_arrival_sec` that's over 24 hours.

In [ ]:
more_than_86400 = trips_routes_times2.loc[
    trips_routes_times2["scheduled_arrival_sec"] > 86_400
]

In [ ]:
more_than_86400["scheduled_arrival_sec"].describe()

In [ ]:
# more_than_86400[["service_date", "rt_arrival_sec", "scheduled_arrival_sec]].head()

In [ ]:
timestamp_subset = [
    "converted_schd_arrival",
    "converted_rt_arrival",
    "scheduled_arrival_sec",
    "rt_arrival_sec",
    "service_date",
]

#### Subset to make the df smaller


In [ ]:
subset = [
    "stop_id",
    "stop_sequence",
    "scheduled_arrival_sec",
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "rt_arrival_sec",
    "route_id",
    "shape_array_key",
    "route_long_name",
    "direction_id",
    "organization_name",
    "caltrans_district",
    "service_date",
    "route_type",
    "feed_key",
]

In [ ]:
trips_routes_times2 = trips_routes_times2[subset]

In [ ]:
trips_routes_times2.scheduled_arrival_sec= trips_routes_times2.scheduled_arrival_sec.fillna(0)

In [ ]:
trips_routes_times2["converted_rt_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["rt_arrival_sec"] % 86400, unit="s")

In [ ]:
trips_routes_times2["converted_schd_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["scheduled_arrival_sec"] % 86400, unit="s")

In [ ]:
trips_routes_times2.converted_schd_arrival.describe()

#### Checkout results from the code
* Make sure it makes sense.


In [ ]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] > 86_400][
    [
        "scheduled_arrival_sec",
        "converted_schd_arrival",
        "rt_arrival_sec",
        "converted_rt_arrival",
    ]
].sample(20)

### Deal with delays
* Some very extreme values.

In [ ]:
percentiles = [0.01, 0.02, 0.05, 0.1, 0.9, 0.95, 0.98, 0.99]

In [ ]:
trips_routes_times2["delay_min"] = (
        trips_routes_times2["converted_rt_arrival"] - trips_routes_times2["converted_schd_arrival"]
    ).dt.total_seconds() / 60

In [ ]:
display(trips_routes_times2["delay_min"].describe(percentiles))

#### Subtract one day off of `converted_rt_arrival` for these rows.

In [ ]:
trips_routes_times2.loc[trips_routes_times2.delay_min > 1438].sample(3)

In [ ]:
len(trips_routes_times2.loc[trips_routes_times2.delay_min > 1438])

In [ ]:
adjust_rt_time_df = trips_routes_times2.loc[trips_routes_times2.delay_min > 1438].reset_index()

In [ ]:
adjust_rt_time_df['converted_rt_arrival'] = adjust_rt_time_df['converted_rt_arrival'] - pd.Timedelta(days=1)

#### Subtract one day off `converted_schd_arrival`

In [ ]:
trips_routes_times2.loc[trips_routes_times2.delay_min < -1438].sample(3)

In [ ]:
adjust_schd_time_df = trips_routes_times2.loc[trips_routes_times2.delay_min < -1438].reset_index()

In [ ]:
adjust_schd_time_df['converted_schd_arrival'] = adjust_schd_time_df['converted_schd_arrival'] - pd.Timedelta(days=1)

In [ ]:
len(adjust_schd_time_df)

### Add back corrected rows.
#### Get rid of old rows.

In [ ]:
trips_routes_times3 = trips_routes_times2.loc[(trips_routes_times2.delay_min > -1438)].reset_index()

In [ ]:
trips_routes_times3 = trips_routes_times3.loc[(trips_routes_times3.delay_min < 1438)].reset_index()

#### Merge

In [ ]:
m1 = pd.merge(trips_routes_times3, adjust_rt_time_df, how = "left")

In [ ]:
m1.head(1)

In [ ]:
m1.shape

In [ ]:
m2 = pd.merge(m1, adjust_schd_time_df, how = "left")

In [ ]:
len(trips_routes_times2)

In [ ]:
len(trips_routes_times3)

#### Sort again

In [ ]:
trips_routes_times3 = trips_routes_times3.sort_values(
    by=[
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "shape_array_key",
        "direction_id",
        "stop_sequence",
        "stop_id",
        "rt_arrival_sec",
    ]
).reset_index(drop=True)

### Question: Last time, I received the suggestion to throw away things more than 2 hrs because that is not bunching. But wouldn't this be misleading as a metric when we want to calculate all the rows?

In [ ]:
print(trips_routes_times2.delay_min.describe(percentiles))

In [ ]:
trips_routes_times2.converted_schd_arrival.describe()

In [ ]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min < 120
# ].reset_index(drop=True)

In [ ]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min > -120
# ].reset_index(drop=True)

### Calculate the actual & scheduled headway the `operator-route-direction_id-stop_sequence-stop_id-` grain


In [ ]:
groupby_cols = [
    "schedule_gtfs_dataset_key",
    "route_long_name",
    "direction_id",
    "stop_id",
    "stop_sequence",
]

In [ ]:
trips_routes_times2["actual_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_rt_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [ ]:
trips_routes_times2["scheduled_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_schd_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [ ]:
trips_routes_times2["scheduled_arrival_lag_min"].describe(percentiles)

In [ ]:
trips_routes_times2["actual_arrival_lag_min"].describe(percentiles)

In [ ]:
preview_time_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "stop_id",
    "stop_sequence",
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_id == "3961")
][preview_time_col]

### Many lags are actually empty b/c it's the first of that groupby-sequence.

In [ ]:
trips_routes_times2[trips_routes_times2["scheduled_arrival_lag_min"].isna()][
    preview_time_col
].sample()

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 2)
][preview_time_col]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 59)
][preview_time_col]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 46)
    & (trips_routes_times2.stop_id == "5685")
][preview_time_col]

In [ ]:
# rt_stop_times4 = rt_stop_times4.fillna(0)

### Transit Matters Method

In [ ]:
transit_matters_df1 = trips_routes_times2.copy()

In [ ]:
transit_matters_df1["pct_actual_schd_headway"] = (
    transit_matters_df1.actual_arrival_lag_min
    / transit_matters_df1.scheduled_arrival_lag_min
)

In [ ]:
transit_matters_df1["bunched_y_n"] = np.where(
    transit_matters_df1["pct_actual_schd_headway"] < 0.25, "bunched", "not bunched"
)

In [ ]:
transit_matters_df1.bunched_y_n.value_counts() / len(transit_matters_df1)

#### Aggregate.
* At this point, it doesn't matter the sequence, we just care about how bunched the traffic is around one partiuclar stop. 
* See how many trips for that grain are considered "bunched" or not.

In [ ]:
def bunched_not_bunched(
    df: pd.DataFrame, bunched_y_n: str, groupby_cols: list
) -> pd.DataFrame:
    df2 = df.loc[df.bunched_y_n == bunched_y_n].reset_index(drop=True)

    bunched_y_n = bunched_y_n.replace(" ", "_")
    agg1 = (
        df2.groupby(groupby_cols).agg({"trip_instance_key": "nunique"}).reset_index()
    ).rename(columns={"trip_instance_key": f"{bunched_y_n}_trips"})
    return agg1

In [ ]:
def agg_final_df(df: pd.DataFrame) -> pd.DataFrame:
    groupby_cols = [
        "caltrans_district",
        "schedule_gtfs_dataset_key",
        "feed_key",
        "organization_name",
        "route_long_name",
        "route_type",
        "route_id",
        "direction_id",
        "stop_id",
        "stop_sequence",
    ]

    # Find total trips that are bunched
    bunched = bunched_not_bunched(df, "bunched", groupby_cols)

    # Find total trips that are NOT bunched
    not_bunched = bunched_not_bunched(df, "not bunched", groupby_cols)

    # Merge
    m1 = pd.merge(not_bunched, bunched, on=groupby_cols, how="outer")

    # Find the % of bunched trips
    m1 = m1.fillna(0)
    m1["all_trips"] = m1.not_bunched_trips + m1.bunched_trips
    m1["per_trip_bunched_per_stop"] = m1.bunched_trips / m1.all_trips

    # Filter out any rows with only one trip of that groupby combo
    # for that service date
    m1 = m1.loc[m1.all_trips > 1].reset_index(drop=True)
    m1 = m1.drop(columns=["not_bunched_trips", "bunched_trips"])

    return m1

In [ ]:
transit_matters_m1 = agg_final_df(transit_matters_df1)

In [ ]:
# transit_matters_m1 = (
#   transit_matters_m1.sort_values(by=["all_trips"], ascending=False)
#   .drop_duplicates(subset=transit_matters_agg)
#   .reset_index(drop=True)
# )

### Help: Swapped order of a bus is messing with the transit matters metric.
* How to solve for this?? 

In [ ]:
preview_cols = [
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
    "pct_actual_schd_headway",
    "bunched_y_n",
]

In [ ]:
example2 = transit_matters_df1.loc[
    (transit_matters_df1.stop_id == "2307719")
    & (transit_matters_df1.organization_name == "City of Visalia")
    & (transit_matters_df1.route_id == "2042")
    & (transit_matters_df1.shape_array_key == "60da59c7000ea5dcb5f845d8fa227f14")
]

#### Starting row 33484: the RT Arrival time is swapped. A bus that was scheduled to arrive at 4:27 arrived boefre the bus arrived at 3:42.
* This repeats again row 33486.

In [ ]:
example2[preview_cols]

### Use 2 minute benchmark
* [Source](https://static1.squarespace.com/static/533b9a24e4b01d79d0ae4376/t/645e82de1f570b31497c44dc/1683915486889/TransitMatters-Headwaymanagement.pdf)
* Justifying the use of
headway maintenance. For example, in April
2022 the 66 bus significantly bunched around
several stops. When bunching is defined as
buses that run within two minutes or less of
each other, inbound buses towards Nubian
Square bunched 10% of the time at Brigham
Circle, 9% at Brookline Village and Roxbury
Crossing, and 8% of the time at Coolidge
Corner. Bunching is even more dramatic
outbound towards Harvard Square where
buses bunched over 35% of the time at Winship
St, 13% at Coolidge Corner and Harvard Ave at
Commonwealth Ave, and 12% at North Harvard
St at Western Ave. View more data about bus
bunching through the TransitMatters Data
Dashboard here.

* To Do: add back in route  & operator information

In [ ]:
two_minutes_df = trips_routes_times2.copy()

In [ ]:
two_minutes_df["bunched_y_n"] = np.where(
    two_minutes_df["actual_arrival_lag_min"] <= 2, "bunched", "not bunched"
)

In [ ]:
two_minutes_df.bunched_y_n.value_counts()

In [ ]:
final_two_minute = agg_final_df(two_minutes_df)

In [ ]:
final_two_minute.loc[
    (final_two_minute.stop_id == "2307695")
    & (final_two_minute.organization_name == "City of Visalia")
    & (final_two_minute.route_id == "2042")
]

### Comparing both outcomes
* There are so many more bunched trips for the 2 minute approach.
* Add back in schedule_gtfs_key and then grab stop level data from the warehouse.

In [ ]:
final_two_minute.per_trip_bunched_per_stop.describe(percentiles)

In [ ]:
transit_matters_m1.per_trip_bunched_per_stop.describe(percentiles)

### Make Visuals

In [ ]:
freq_range = [
    "#ccbb44",
    "#e9d868",
    "#fcb40e",
    "#ff9c42",
    "#fc5c04",
    "#dd217d",
    "#dd217d",
    "#dd217d",
]

In [ ]:
trips_routes_times2["hour"] = trips_routes_times2["converted_rt_arrival"].dt.hour
trips_routes_times2["min"] = trips_routes_times2["converted_rt_arrival"].dt.minute

In [ ]:
trips_routes_times2.head(2)

In [ ]:
def compare_approaches(
    stop_id: str, organization_name: str, route_id: str, stop_sequence: int
):
    transit_matter = transit_matters_m1.loc[
        (transit_matters_m1.stop_id == stop_id)
        & (transit_matters_m1.organization_name == organization_name)
        & (transit_matters_m1.route_id == route_id)
        & (transit_matters_m1.stop_sequence == stop_sequence)
    ]
    display(transit_matter)

    two_min = final_two_minute.loc[
        (final_two_minute.stop_id == stop_id)
        & (final_two_minute.organization_name == organization_name)
        & (final_two_minute.route_id == route_id)
        & (final_two_minute.stop_sequence == stop_sequence)
    ]

    display(two_min)
    total_trips = trips_routes_times2.loc[
        (trips_routes_times2.stop_id == stop_id)
        & (trips_routes_times2.organization_name == organization_name)
        & (trips_routes_times2.route_id == route_id)
        & (trips_routes_times2.stop_sequence == stop_sequence)
    ]

    display(total_trips.trip_instance_key.nunique())

    chart = (
        alt.Chart(total_trips)
        .mark_circle(size=500)
        .encode(
            x="hour",
            y="min",
            color=alt.Color(
                "hour",
                scale=alt.Scale(range=freq_range),
            ),
            tooltip=["hour", "min", "actual_arrival_lag_min"],
        )
        .properties(width=800, height=400)
    )
    display(chart)
    return total_trips

In [ ]:
test1 = compare_approaches(
    stop_id="5685",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
    stop_sequence=46,
)

In [ ]:
test2 = compare_approaches(
    stop_id="2307469",
    organization_name="City of Visalia",
    route_id="2042",
    stop_sequence=27,
)

In [ ]:
test3 = compare_approaches(
    stop_id="3104",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=80,
)

In [ ]:
test4 = compare_approaches(
    stop_id="15320",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=64,
)

In [ ]:
test5 = compare_approaches(
    stop_id="3288014",
    organization_name="City of Thousand Oaks",
    route_id="3402",
    stop_sequence=16,
)

In [ ]:
transit_matters_m1.sort_values(by=["per_trip_bunched_per_stop"], ascending=False)

### Make Maps 
* Think I actually need the vehicle positions since stops are literally the stop's geometry, so it'll always be plotting on the same spot.

* https://github.com/cal-itp/data-analyses/blob/db19b70329f1e817236bda13707dd903c24abb4c/_shared_utils/shared_utils/gtfs_utils_v2.py#L371
* https://github.com/cal-itp/data-analyses/blob/main/gtfs_funnel/download_stops.py

In [ ]:
stop

In [ ]:
# What is this file?
vps_gdf = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/vp_2024-05-22.parquet"
)

In [ ]:
vps_gdf = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
        "geometry",
    ]
]

In [ ]:
vps_df = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
    ]
]

In [ ]:
vps_m1 = pd.merge(
    vps_df,
    trips_routes_times2,
    on=["schedule_gtfs_dataset_key", "trip_instance_key"],
    how="inner",
)

In [ ]:
def one_stop(df: pd.DataFrame, stop_id: str, org_name: str, route_id: str):
    # Look at one route & stop
    test_route1 = df.loc[
        (df.organization_name == org_name)
        & (df.route_id == route_id)
        & (df.stop_id == stop_id)
    ]

    compare_approaches(stop_id=stop_id, organization_name=org_name, route_id=route_id)

    # display(test_route1.explore("time_int", marker_kwds = {'radius':20}))
    return test_route1

In [ ]:
metro_test1 = one_stop(
    vps_m1,
    stop_id="5700",
    org_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
)

In [ ]:
metro_test1.shape

In [ ]:
metro_test1.sample(1)

### Other
* https://www.sciencedirect.com/science/article/pii/S1366554523003666
* https://www.sciencedirect.com/science/article/pii/S0968090X22002492?ref=pdf_download&fr=RR-2&rr=8d7d6fb73d8015be